<a href="https://colab.research.google.com/github/astha77-bot/Play-store-reviews-scrapper/blob/main/reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq google-play-scraper

     |████████████████████████████████| 51kB 2.8MB/s 


In [ ]:
!pip install -qq -U watermark

In [ ]:
#for apps logos(if required)
%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper

Python implementation: CPython
Python version       : 3.7.10
IPython version      : 5.5.0

pandas             : 1.1.5
matplotlib         : 3.2.2
seaborn            : 0.11.1
google_play_scraper: 0.1.2



In [ ]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
#Python package to highlight code snippets.
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app
#seaborn and matplotlib are used for visualisation
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [ ]:
app_packages=[
'sisinc.com.sis'
]

In [ ]:
len(app_packages)

1

In [ ]:
app_infos = []
#user country as well as language you're looking for
#for loop as if you want multiple app packages
for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  5.84it/s]


In [ ]:
  #scrapper will generate very unsystematics reviews format,so use jsonlexer from pygments package


In [ ]:

def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [ ]:
print_json(app_infos[0])

{
  "adSupported": true,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 and up",
  "appId": "sisinc.com.sis",
  "containsAds": true,
  "contentRating": "Mature 17+",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "Meme Chat is the ultimate social networking app to revolutionize the world of memes.  Its a social networking app made for meme lovers and meme makers around the world!\r\n\r\nDon\u2019t we love watching and sharing memes? Why not have a community for it!\r\nPerfect entertainment best known for curing depression, sadness and will fulfill your greed for humor. Join Meme Chat and start uploading funny memes on this one of its kind meme social network.\r\n\r\nMeme Social Network:\r\nMeme Chat is going to be your next favorite social media network. You can get registered on the app and choose from various categories of your own interests and connect with millions of meme lovers from around the globe. \r\n\r\nTry Out Meme Keyboard:\r\nActivate t

In [ ]:
def format_title(title):
  sep_index = title.find(':') if title.find(':') != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

fig, axs = plt.subplots(2, len(app_infos) // 2, figsize=(14, 5))

for i, ax in enumerate(axs.flat):
  ai = app_infos[i]
  img = plt.imread(ai['icon'])
  ax.imshow(img)
  ax.set_title(format_title(ai['title']))
  ax.axis('off')

<Figure size 1008x360 with 0 Axes>

In [ ]:
app_infos_df=pd.DataFrame(app_infos)
app_infos_df.head()

,title,description,descriptionHTML,summary,summaryHTML,installs,minInstalls,score,ratings,reviews,histogram,price,free,currency,sale,saleTime,originalPrice,saleText,offersIAP,inAppProductPrice,size,androidVersion,androidVersionText,developer,developerId,developerEmail,developerWebsite,developerAddress,privacyPolicy,developerInternalID,genre,genreId,icon,headerImage,screenshots,video,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,editorsChoice,appId,url
0,"MemeChat - Memes, Keyboard, Trends & News",Meme Chat is the ultimate social networking ap...,Meme Chat is the ultimate social networking ap...,"Make Memes, Make Money & Make Communities - In...","Make Memes, Make Money &amp; Make Communities ...","1,000,000+",1000000,3.819422,18148,7206,"[4129, 613, 980, 1109, 11317]",0,True,USD,False,None,None,None,False,None,66M,5.0,5.0 and up,Meme Chat Pvt. Ltd.,Meme+Chat+Pvt.+Ltd.,contact@memechat.app,http://memechat.app/,None,https://memechat.app/privacy.html,9035387236720478977,Entertainment,ENTERTAINMENT,https://play-lh.googleusercontent.com/oUkmCh_d...,https://play-lh.googleusercontent.com/Dp0t9s5r...,[https://play-lh.googleusercontent.com/WkOg52Z...,None,None,Mature 17+,None,True,True,"Jul 5, 2017",1609507895,5.02,"NEW YEAR, NEW UPDATE: A LOT OF NEW THINGS ARE ...","NEW YEAR, NEW UPDATE: A LOT OF NEW THINGS ARE ...",False,sisinc.com.sis,https://play.google.com/store/apps/details?id=...


In [ ]:
#he app information for later by converting the JSON objects into a Pandas dataframe as to  save the result into a CSV file:

In [ ]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

In [ ]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.RATING]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 30000 if score == 3 else 30000,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'rating'
        r['appId'] = ap
      app_reviews.extend(rvs)
       

100%|██████████| 1/1 [00:07<00:00,  7.32s/it]


In [ ]:

print_json(app_reviews[0])


{
  "appId": "sisinc.com.sis",
  "at": "2020-05-16 09:26:38",
  "content": "Faltu app hai aur devloper corrupted ,paise ka lalch dekar 5 star dene ko khte hain users se. Mtlb ki inko khud ko pta hai ki bakwas app hai ye",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": null,
  "reviewId": "gp:AOqpTOGBbMUyG15Eln0JiGMxiYK4UYsN9B30JMXP3HrU_nkzOofMl30ZDIoR87NmQn3dOVz9UCwwX_xJvg9rGw",
  "score": 1,
  "sortOrder": "rating",
  "thumbsUpCount": 0,
  "userImage": "https://play-lh.googleusercontent.com/-QNiy4SBid8M/AAAAAAAAAAI/AAAAAAAAAAA/AMZuucnDA84uMCR1h48ROjU_wc4AnKffNQ/photo.jpg",
  "userName": "Ihtesham Faisal"
}



In [ ]:

len(app_reviews)

7194

In [ ]:

app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)